# Combining MQTT and STMPY

In this notebook, we want to combine MQTT with a STMPY state machine. For just getting this first connection implemented, the result does nothing really useful, but should be easy to run and debug. I promise, the system we are going to build next week based on what we learn this week will be more useful, but this way we can learn one thing at a time.

The application is simple:

 * We subscribe to an MQTT topic.
 * Whenever we receive a message at that topic, we trigger an event `message` in the state machine.
 * The state machine just switches between to states `tick` and `tock`.
 * Whenver the state machine switches, it also publishes an MQTT message that show the number of ticks and tocks.
 
Here is the diagram of the machine:

![](images/mqtt-tick-tock.png)

Let us first declare this machine, in the same way we did it before in STMPY:

In [1]:
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random

class Quizmaster:
    def __init__(self):
        print("Init!")
        self.button_ask = widgets.Button(description="Ask")
        self.button_ask.on_click(self.on_button_ask)
        self.answered = ""
        self.message = ""
        # text field
        self.text = widgets.Text(value='', placeholder='', description='String:', disabled=False)
        display(self.text, self.button_ask)
    
    def on_button_ask(self, b):
        self.stm.send('ask') # <---- here we send a message
        print(self.text.value)
        
    def on_idle(self):
        print("idle")
        self.text.set_trait(name='value', value='idle...')

    def on_asking(self):
        print("asking")
        self.text.set_trait(name='value', value='asking!')
        
    def on_received(self, message):
        print("received")
        self.answered = self.message
        self.text.set_trait(name='value', value=f"{self.message}")
    
    def send_mqtt_tick(self):
        print("Tick! {}".format(self.ticks))
        self.ticks = self.ticks + 1
        self.mqtt_client.publish("tick", self.ticks)

    def send_mqtt_tock(self):
        print("Tock! {}".format(self.tocks))
        self.tocks = self.tocks + 1
        self.mqtt_client.publish("tock", self.ticks)

# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions
t1 = {'trigger':'ask', 
      'source':'idle', 
      'target':'asking'}
t2 = {'trigger':'t', 
      'source':'asking', 
      'target':'idle'}
t3 = {'trigger':'t', 
      'source':'received', 
      'target':'idle'}
t4 = {'trigger':'message', 
      'source':'asking', 
      'target':'received'}
t5 = {'trigger':'reset', 
      'source':'asking', 
      'target':'idle'}

 
# the states:
idle = {'name': 'idle',
       'entry': 'on_idle'}

asking = {'name': 'asking',
        'entry': 'on_asking; start_timer("t", 20000)'}

received = {'name': 'received',
        'entry': 'on_received; stop_timer("t")'}



The coupling with MQTT is only visible at a few points:

 * We use the line `self.mqtt_client.publish('tick', self.ticks)` to publish the MQTT message. We just need to remember to set the variable `self.mqtt_client` with the Paho MQTT client. (Note that we here use the Paho client direclty, not our wrapper class MQTT_Client_1).
 * The state machine is triggered by the message `message`. This will be dispatched by the MQTT client.

In [2]:
from threading import Thread

import paho.mqtt.client as mqtt


class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        message = json.loads(msg.payload.decode('utf-8'))["msg"]
        self.driver.message = message
        print("on_message(): topic: {}".format(msg.topic))
        self.stm_driver.send("message", "quiz_master")
        
        

    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("team2")

        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

Our MQTT client above is now configured so that whenever it receives an MQTT message, it dispatches a message into the state machine driver, with the `tick_tock` machine as address, so that it is sent to the state machine. There, it triggers the transitions.

In the following, we start the MQTT client and the state machine driver, connect them with each other, and start everything:

In [4]:
# broker, port = 'iot.eclipse.org', 1883
broker, port = "mqtt20.iik.ntnu.no", 1883

quiz_master = Quizmaster()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5], states=[idle, asking, received], obj=quiz_master, name="quiz_master")
quiz_master.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(quiz_master)
quiz_master.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(broker, port)

Init!


Text(value='', description='String:', placeholder='')

Button(description='Ask', style=ButtonStyle())

idleConnecting to mqtt20.iik.ntnu.no:1883

on_connect(): Connection Accepted.
